<a href="https://colab.research.google.com/github/meta-llama/llama-cookbook/blob/main/end-to-end-use-cases/coding/text2sql/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>  

## Quick Demo of Text2SQL Using Llama 3.3

This demo shows how to use Llama 3.3 to answer questions about a SQLite DB. 

We'll use LangChain and the Llama cloud provider [Together.ai](https://api.together.ai/), where you can easily get a free API key (or you can use any other Llama cloud provider or even Ollama running Llama locally).

In [ ]:
!pip install --upgrade -r requirements.txt

In [2]:
import os
from langchain_together import ChatTogether

os.environ['TOGETHER_API_KEY'] = 'your_api_key'

llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
    temperature=0,
)

To recreate the `nba_roster.db` file, run the two commands below:
- `python txt2csv.py` to convert the `nba.txt` file to `nba_roster.csv`. The `nba.txt` file was created by scraping the NBA roster info from the web.
- `python csv2db.py` to convert `nba_roster.csv` to `nba_roster.db`.

In [ ]:
# uncomment if you don't want to create the db yourself
#! wget https://github.com/meta-llama/llama-recipes/raw/3649841b426999fdc61c30a9fc8721106bec769b/recipes/use_cases/coding/text2sql/nba_roster.db

In [4]:
from langchain_community.utilities import SQLDatabase

# Note: to run in Colab, you need to upload the nba_roster.db file in the repo to the Colab folder first.
db = SQLDatabase.from_uri("sqlite:///nba_roster.db", sample_rows_in_table_info=0)

def get_schema():
    return db.get_table_info()

In [5]:
question = "What team is Stephen Curry on?"
prompt = f"""Based on the table schema below, write a SQL query that would answer the user's question; just return the SQL query and nothing else.

Scheme:
{get_schema()}

Question: {question}

SQL Query:"""

print(prompt)

Based on the table schema below, write a SQL query that would answer the user's question; just return the SQL query and nothing else.

Scheme:

CREATE TABLE nba_roster (
	"Team" TEXT, 
	"NAME" TEXT, 
	"Jersey" TEXT, 
	"POS" TEXT, 
	"AGE" INTEGER, 
	"HT" TEXT, 
	"WT" TEXT, 
	"COLLEGE" TEXT, 
	"SALARY" TEXT
)

Question: What team is Stephen Curry on?

SQL Query:


In [6]:
answer = llm.invoke(prompt).content
print(answer)

SELECT Team FROM nba_roster WHERE NAME = 'Stephen Curry'


***Note:*** If you don't have the "just return the SQL query and nothing else" in the prompt above, you'll likely get more text other than the SQL query back in the answer, making some extra post-processing necessary before running the db query below.

In [7]:
# note this is a dangerous operation and for demo purpose only; in production app you'll need to safe-guard any DB operation
result = db.run(answer)
result

"[('Golden State Warriors',)]"

In [8]:
# how about a follow up question
follow_up = "What's his salary?"
print(llm.invoke(follow_up).content)

I don't have enough information to determine whose salary you are referring to. Could you please provide more context or specify the person you are asking about?


Since we did not pass any context along with the follow-up to Llama, it doesn't know the answer. Let's try to fix it by adding context to the follow-up prompt.

In [9]:
prompt = f"""Based on the table schema, question, SQL query, and SQL response below, write a new SQL response; be concise, just output the SQL response.

Scheme:
{get_schema()}

Question: {follow_up}
SQL Query: {question}
SQL Result: {result}

New SQL Response:
"""
print(prompt)

Based on the table schema, question, SQL query, and SQL response below, write a new SQL response; be concise, just output the SQL response.

Scheme:

CREATE TABLE nba_roster (
	"Team" TEXT, 
	"NAME" TEXT, 
	"Jersey" TEXT, 
	"POS" TEXT, 
	"AGE" INTEGER, 
	"HT" TEXT, 
	"WT" TEXT, 
	"COLLEGE" TEXT, 
	"SALARY" TEXT
)

Question: What's his salary?
SQL Query: What team is Stephen Curry on?
SQL Result: [('Golden State Warriors',)]

New SQL Response:



In [10]:
new_answer = llm.invoke(prompt).content
print(new_answer)

SELECT SALARY FROM nba_roster WHERE NAME = "Stephen Curry"


Because we have "be concise, just output the SQL response", Llama 3 is able to just generate the SQL statement; otherwise output parsing will be needed.

In [11]:
db.run(new_answer)

"[('$51,915,615',)]"